In [1]:
import os
import ee
import geemap
import ipywidgets as widgets
from ipyleaflet import WidgetControl
from bqplot import pyplot as plt
from fpdf import FPDF
from datetime import date
import numpy as np
from jenkspy import JenksNaturalBreaks

In [2]:
Map = geemap.Map(center=(10, 40), zoom=2)

In [3]:
Map

Map(center=[10, 40], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [4]:
style = {'description_width': 'initial'}
layout = widgets.Layout(width='95%')

geom_options = ['Uploaded vector', 
              'Drawn vector']
geom_widget = widgets.Dropdown(options=geom_options, value='Uploaded vector', description='AOI type:', style=style)

full_widget = widgets.VBox([
    widgets.HBox([geom_widget]),
])

full_widget

In [5]:
widg_cal = widgets.IntRangeSlider(
    value=[2015, 2018],
    min=2000,
    max=2021,
    step=1,
    description='Calibration period:',
    orientation='horizontal',
    layout=layout, 
    style=style
)

widg_val = widgets.IntRangeSlider(
    value=[2019, 2021],
    min=2000,
    max=2021,
    step=1,
    description='Validation period:',
    orientation='horizontal',
    layout=layout, 
    style=style
)

widg_hansen = widgets.IntSlider(
    min=1, 
    max=99, 
    value=10, 
    description='Tree cover threshold (%)>:', 
    layout=layout, 
    style=style
)

widg_percentile = widgets.IntSlider(
    min=1, 
    max=99, 
    value=99, 
    description='Insignificant Risk distance percentile:', 
    layout=layout, 
    style=style
)

widg_class1 = widgets.IntText(
    value=600,
    description='Class 1 window size:',
    style=style
)

widg_class2 = widgets.IntText(
    value=1200,
    description='Class 2 window size:',
    style=style
)
widg_class3 = widgets.IntText(
    value=1800,
    description='Class 3 window size:',
    style=style
)
widg_class4 = widgets.IntText(
    value=2400,
    description='Class 4 window size:',
    style=style
)
widg_class5 = widgets.IntText(
    value=3000,
    description='Class 5 window size:',
    style=style
)
widg_class6 = widgets.IntText(
    value=6000,
    description='Class 6 window size:',
    style=style
)
widg_class7 = widgets.IntText(
    value=12000,
    description='Class 7 window size:',
    style=style
)
widg_class8 = widgets.IntText(
    value=18000,
    description='Class 8 window size:',
    style=style
)
widg_class9 = widgets.IntText(
    value=24000,
    description='Class 9 window size:',
    style=style
)
widg_class10 = widgets.IntText(
    value=30000,
    description='Class 10 window size:',
    style=style
)
widg_res = widgets.IntText(
    value=30,
    description='Pixel resolution:',
    style=style
)
widg_gridres = widgets.IntText(
    value=10000,
    description='Grid resolution:',
    style=style
)
widg_classmeth = widgets.Dropdown(
    options=['Equal Area', 'Equal Interval', 'Jenks'],
    value='Equal Area',
    description='Classification methodology:',
    style=style
)


hbox_classwidg1 = widgets.HBox([widg_class1,widg_class2,widg_class3,widg_class4])
hbox_classwidg2 = widgets.HBox([widg_class5,widg_class6,widg_class7,widg_class8])
hbox_classwidg3 = widgets.HBox([widg_class9,widg_class10])
hbox_classwidg4 = widgets.HBox([widg_res,widg_gridres,widg_classmeth])

vbox_widg = widgets.VBox([widg_cal,widg_val,widg_hansen,widg_percentile,hbox_classwidg1,hbox_classwidg2,hbox_classwidg3,hbox_classwidg4])
vbox_widg

In [6]:
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

In [8]:
style = {'description_width': 'initial'}
uploader = widgets.FileUpload(
    description='Upload data',
    accept='.zip, .json, .geojson', 
    multiple=False,
    button_style='primary',
    style=style
)

submit = widgets.Button(
    description='Display data',
    button_style='success',
    tooltip='Click me',
    style=style)

reset = widgets.Button(
    description='Reset',
    button_style='warning',
    tooltip='Click me',
    style=style)

submit_Step1 = widgets.Button(
    description='Run Step 1',
    button_style='primary',
    tooltip='Click me',
    style=style
)

submit_Step2 = widgets.Button(
    description='Run Step 2',
    button_style='primary',
    tooltip='Click me',
    style=style
)

submit_Step3 = widgets.Button(
    description='Run Step 3',
    button_style='primary',
    tooltip='Click me',
    style=style
)

submit_Step4 = widgets.Button(
    description='Run Step 4',
    button_style='primary',
    tooltip='Click me',
    style=style
)

button_widget = widgets.HBox([
    submit_Step1,submit_Step4
])

button_widget

In [9]:
layer_list_widgets  = widgets.VBox([])

layer_accordion = widgets.Accordion(children=[layer_list_widgets],selected_index=None)
layer_accordion.set_title(0, 'Layers')

layer_accordion

Accordion(children=(VBox(),), selected_index=None, _titles={'0': 'Layers'})

In [10]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads/cambodia_MoE14_EA_WORKSPACE')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

pdf_link = os.path.join(out_dir, 'report.pdf') 

In [11]:
with output_widget:
    print('Upload shapefile or \ngeojson as a zip file')
    display(uploader)
    display(submit)
    display(reset)

In [12]:
def get_vector(upload_widget, out_dir=None):
    
    import zipfile
    import glob
    
    if out_dir is None:
        out_dir = os.path.join(os.path.expanduser('~'), 'Downloads/cambodia_MoE14_EA_WORKSPACE')
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
        
    vector = None
    
    try:    
        [uploaded_file] = upload_widget.value
        file = upload_widget.value[uploaded_file]
        name = file['metadata']['name']
        content = file['content']
        out_file = os.path.join(out_dir, name)
        with open(out_file, "wb") as fp:
            fp.write(content)

        if name.endswith('.zip'):
            with zipfile.ZipFile(out_file, "r") as zip_ref:
                extract_dir = os.path.join(out_dir, name[:-4] + "_" + geemap.random_string(3))
                zip_ref.extractall(extract_dir)
                files = glob.glob(extract_dir + '/*.shp')
                if len(files) > 0:
                    shp = files[0]
                    vector = geemap.shp_to_ee(shp)
                else:
                    files = glob.glob(extract_dir + '/*.geojson')
                    if len(files) > 0:
                        geojson = files[0]
                        vector = geemap.geojson_to_ee(geojson)
        else:
            vector = geemap.geojson_to_ee(out_file)
                                
    except Exception as e:
        print(e)

    return vector

In [13]:
def submit_clicked(b):
    if uploader._counter > 0:
        Map.default_style = {'cursor': 'wait'}        
        try:
            fc = get_vector(uploader)
            layer_name = 'Layer ' + geemap.random_string(3)
            Map.addLayer(fc, {}, layer_name)
            Map.centerObject(fc)
            uploader.value.clear()
            uploader._counter = 0    
        except Exception as e:
            print(e)
        Map.default_style = {'cursor': 'pointer'}

submit.on_click(submit_clicked)

def reset_clicked(b):

    Map.layers = Map.layers[:3]
    output_widget.clear_output()
    with output_widget:
        print('Upload shapefile or \ngeojson as a zip file')
        display(uploader)
        display(submit)
        display(reset)
    uploader.value.clear()
    uploader._counter = 0
    
reset.on_click(reset_clicked)

In [14]:
download_widget = widgets.Output(layout={'border': '1px solid black'})
download_widget.clear_output()
download_widget

Output(layout=Layout(border='1px solid black'))

In [15]:
status_widget = widgets.Output(layout={'border': '1px solid black'})
status_widget.clear_output()
status_widget_box = widgets.HBox([status_widget], layout=widgets.Layout(height='1000px', width='300px',overflow_y='scroll'))
status_widget_box

In [16]:
children_layers = []
tab_layers = widgets.Tab()
tab_layers.children = children_layers

list_widgets_layers  = widgets.VBox([tab_layers])
layer_list_widgets.children =  tuple(list(layer_list_widgets.children) + [list_widgets_layers]) 

In [17]:
def geomX():
    if geom_widget.value == 'Uploaded vector':
        geomX = ee.FeatureCollection(get_vector(uploader)).first().geometry()
    elif geom_widget.value == 'Drawn vector':
        geomX = ee.Feature(ee.List(Map.draw_features).get(0)).geometry()
        
    return geomX

def geomX_rasterzed():
    fc_geomX = ee.FeatureCollection([ee.Feature(geomX(),{'foo':1})])
    
    return fc_geomX.reduceToImage(['foo'], ee.Reducer.sum()).gt(0)

def hansen_singleyear(year):
    year = ee.Number(year).subtract(2000)
    
    hansen_precambo = ee.Image("UMD/hansen/global_forest_change_2022_v1_10")
    
    # cambo specific
    
    MoE_LC_2014 = ee.Image('projects/carbondevelopmentteam/assets/Cambodia/National/MoE_LC_2014').unmask(0)


    forest_MoE = MoE_LC_2014.eq(1)\
                  .add(MoE_LC_2014.eq(2))\
                  .add(MoE_LC_2014.eq(3))\
                  .add(MoE_LC_2014.eq(6))\
                  .add(MoE_LC_2014.eq(7))\
                  .add(MoE_LC_2014.eq(9))\
                  .add(MoE_LC_2014.eq(10))\
                  .add(MoE_LC_2014.eq(12))\
                  .add(MoE_LC_2014.eq(14))
    #.add(MoE_LC_2014.eq(11))\ # not including "Pine tree" as it's not in the spreadsheet and has EF of 0 anyway.

    hansen = hansen_precambo.multiply(forest_MoE)

    hansen_tc_2000 = hansen.select('treecover2000').gte(widg_hansen.value)
    hansen_lossyear = hansen.select('lossyear').unmask(0)

    hansen_defor_2001_xx = hansen_lossyear.gt(0).multiply(hansen_lossyear.lte(year))

    return hansen_tc_2000.multiply(hansen_defor_2001_xx.eq(0)).unmask(0).updateMask(geomX_rasterzed())

def historical_deforestation(): 
    return hansen_singleyear(widg_cal.value[0]).subtract(hansen_singleyear(widg_cal.value[1])).gt(0)

def historical_deforestation_NEW(): 
    return hansen_singleyear(widg_cal.value[0]).subtract(hansen_singleyear(widg_val.value[1])).gt(0)

def historical_deforestation_validation(): 
    return hansen_singleyear(widg_val.value[0]).subtract(hansen_singleyear(widg_val.value[1])).gt(0)

def defor_window(res):
    # Circle of 10 x 30m = 600m
    # scale of 600 should be 30m. so we divide by 20.
    # now it's correct at 30m... but what about other resolutions? 
    scaleX = ee.Number(res).divide(20).multiply(ee.Number(widg_res.value).divide(30))
    return historical_deforestation_NEW().reduceNeighborhood(reducer=ee.Reducer.mean(), kernel= ee.Kernel.circle(10))\
                                                .reproject(scale=scaleX,crs=ee.String('EPSG:4326'))

In [18]:
#print(widg_cal.value[0])

In [19]:
#print(widg_val.value[1])

In [20]:
def submit_Step1_clicked(b):
    Map.addLayer(historical_deforestation_NEW(),{'palette':['green','red']},'Historical deforestation',True,0.0)
    Map.addLayer(defor_window(10),{'palette':['green','yellow','red']},'Historical rate of change - 600m',True,0.0)

    layer_hist = Map.layers[-2]
    widg_hist = layer_hist.interact(opacity=(0, 1, 0.1))
    layer_wind = Map.layers[-1]
    widg_wind = layer_wind.interact(opacity=(0, 1, 0.1))
    
    label_hist = widgets.Label('Historical deforestation')
    label_wind = widgets.Label('Historical rate of change - 600m')

    full_widget = widgets.VBox([
        widgets.HBox([label_hist, widg_hist,label_wind,widg_wind]),
    ])
    
    tab_layers.children = tuple(list(tab_layers.children) + [full_widget])  
    tab_layers.set_title(0, 'Step 1')
    button_widget.children =  tuple(list(button_widget.children) + [submit_Step2]) 
    
submit_Step1.on_click(submit_Step1_clicked)

In [21]:
def dist_edge():
    dist_ed = hansen_singleyear(widg_cal.value[0]).neq(1).fastDistanceTransform().sqrt().multiply(ee.Image.pixelArea().sqrt())
    return dist_ed.reproject(crs=ee.String('EPSG:4326'),scale=widg_res.value).clip(geomX())

def significance_buffer():
    #Determine the distance at which 99% of the deforestation or forest degradation occurred during
    #the historical reference period of the FREL.
    randpoints = ee.FeatureCollection.randomPoints(region=geomX(), points=10000, seed=7)
    
    dist_sampled = dist_edge().updateMask(historical_deforestation_NEW()).sampleRegions(collection=randpoints,geometries=False)

    sampled_pxx = dist_sampled.reduceColumns(reducer=ee.Reducer.percentile([widg_percentile.value]), selectors=['distance'])
    sampled_pxx = ee.Number(sampled_pxx.get(sampled_pxx.keys().get(0)))

    # Create a buffer zone around the edge of the non-forest land area that exists at the beginning of
    #the FREL validity period using the calculated distance.
    significance_buffer = dist_edge().gte(ee.Image.constant(sampled_pxx)).updateMask(historical_deforestation_NEW().eq(0))
    return significance_buffer

In [22]:
def submit_Step2_clicked(b):
    Map.addLayer(dist_edge(),{'palette':['red','orange','yellow','green','aqua','blue'], 'max':1000},'Distance from edge',True,0.0)
    Map.addLayer(significance_buffer(),{},'Areas of insignificant risk',True,0.0)
    
    layer_dist = Map.layers[-2]
    widg_dist = layer_dist.interact(opacity=(0, 1, 0.1))
    layer_sigb = Map.layers[-1]
    widg_sigb = layer_sigb.interact(opacity=(0, 1, 0.1))
    
    label_dist = widgets.Label('Distance from edge')
    label_sigb = widgets.Label('Areas of insignificant risk')

    full_widget = widgets.VBox([
        widgets.HBox([label_dist, widg_dist,label_sigb,widg_sigb]),
    ])
    
    
    tab_layers.children = tuple(list(tab_layers.children) + [full_widget])  
    tab_layers.set_title(1, 'Step 2')
    button_widget.children =  tuple(list(button_widget.children) + [submit_Step3]) 

submit_Step2.on_click(submit_Step2_clicked)

In [23]:
print(widg_cal.value[0])

2015


In [24]:
def classification_input(res):
    # Testing first without "insignificant risk" because that's slow
    #was this correct???
    #input = hansen_singleyear(widg_cal.value[0]).multiply(defor_window(res))#.multiply(land)#.subtract(significance_buffer) <----- this makes it
    #CHANGED... it was incorrect
    
    input = hansen_singleyear(widg_cal.value[0]).multiply(defor_window(res))#.multiply(land)#.subtract(significance_buffer) <----- this makes it

    
    # changed to below for now
    #input = defor_window(res)
    #same
    #Map.addLayer(input,{},'input')
    #Map.addLayer(defor_window(res),{},'defor_window(res)')
    #TEST without below...
    input = input.updateMask(input.neq(0))
    return input

    
def result_EI():
    #NOTE: NOT SURE WHY I DISABLED THIS BUT DOUBLE-CHECK WHAT'S GOING ON
    return classification_input(res)#.multiply(-1).add(1)#.multiply(30).toInt() # double-check this is rounding correctly

def result_EA_OLD(res):
    randpoints = ee.FeatureCollection.randomPoints(region=geomX(), points=10000, seed=7)
    input_EA_sampled = classification_input(res).sampleRegions(collection=randpoints, geometries=False)#.filter(ee.Filter.neq('distance',0)) # distance of 0 is non-forest

    percentile_list =  ee.List.sequence(10, 100, 10)
    sampled_percentiles = input_EA_sampled.reduceColumns(reducer=ee.Reducer.percentile(percentile_list), selectors=['treecover2000'])#.get('p99')

    
    p10 = ee.Number(sampled_percentiles.get('p10'))
    p20 = ee.Number(sampled_percentiles.get('p20'))
    p30 = ee.Number(sampled_percentiles.get('p30'))
    p40 = ee.Number(sampled_percentiles.get('p40'))
    p50 = ee.Number(sampled_percentiles.get('p50'))
    p60 = ee.Number(sampled_percentiles.get('p60'))
    p70 = ee.Number(sampled_percentiles.get('p70'))
    p80 = ee.Number(sampled_percentiles.get('p80'))
    p90 = ee.Number(sampled_percentiles.get('p90'))




    class_0 = classification_input(res).lt(p10)
    class_1 = classification_input(res).gte(p10).multiply(classification_input(res).lt(p20))
    class_2 = classification_input(res).gte(p20).multiply(classification_input(res).lt(p30))
    class_3 = classification_input(res).gte(p30).multiply(classification_input(res).lt(p40))
    class_4 = classification_input(res).gte(p40).multiply(classification_input(res).lt(p50))
    class_5 = classification_input(res).gte(p50).multiply(classification_input(res).lt(p60))
    class_6 = classification_input(res).gte(p60).multiply(classification_input(res).lt(p70))
    class_7 = classification_input(res).gte(p70).multiply(classification_input(res).lt(p80))
    class_8 = classification_input(res).gte(p80).multiply(classification_input(res).lt(p90))
    class_9 = classification_input(res).gte(p90)

    result_EA = class_0.multiply(1) \
                    .add(class_1.multiply(2)) \
                    .add(class_2.multiply(3)) \
                    .add(class_3.multiply(4)) \
                    .add(class_4.multiply(5)) \
                    .add(class_5.multiply(6)) \
                    .add(class_6.multiply(7)) \
                    .add(class_7.multiply(8)) \
                    .add(class_8.multiply(9)) \
                    .add(class_9.multiply(10)) \
                    .unmask(1)\
                    .updateMask(hansen_singleyear(widg_cal.value[0]).eq(1))\
                    .multiply(significance_buffer().neq(1))

    
    with status_widget:
        print('running ' + str(res) + 'm')
    
    return result_EA



In [25]:
def result_EA(res):
    randpoints = ee.FeatureCollection.randomPoints(region=geomX(), points=10000, seed=7)
    input_EA_sampled = classification_input(res).sampleRegions(collection=randpoints, geometries=False)#.filter(ee.Filter.neq('distance',0)) # distance of 0 is non-forest

    
    percentile_list =  ee.List.sequence(10, 100, 10)
    sampled_percentiles = input_EA_sampled.reduceColumns(reducer=ee.Reducer.percentile(percentile_list), selectors=['treecover2000'])#.get('p99')

    
    p10 = ee.Number(sampled_percentiles.get('p10'))
    p20 = ee.Number(sampled_percentiles.get('p20'))
    p30 = ee.Number(sampled_percentiles.get('p30'))
    p40 = ee.Number(sampled_percentiles.get('p40'))
    p50 = ee.Number(sampled_percentiles.get('p50'))
    p60 = ee.Number(sampled_percentiles.get('p60'))
    p70 = ee.Number(sampled_percentiles.get('p70'))
    p80 = ee.Number(sampled_percentiles.get('p80'))
    p90 = ee.Number(sampled_percentiles.get('p90'))




    class_0 = classification_input(res).lt(p10)
    class_1 = classification_input(res).gte(p10).multiply(classification_input(res).lt(p20))
    class_2 = classification_input(res).gte(p20).multiply(classification_input(res).lt(p30))
    class_3 = classification_input(res).gte(p30).multiply(classification_input(res).lt(p40))
    class_4 = classification_input(res).gte(p40).multiply(classification_input(res).lt(p50))
    class_5 = classification_input(res).gte(p50).multiply(classification_input(res).lt(p60))
    class_6 = classification_input(res).gte(p60).multiply(classification_input(res).lt(p70))
    class_7 = classification_input(res).gte(p70).multiply(classification_input(res).lt(p80))
    class_8 = classification_input(res).gte(p80).multiply(classification_input(res).lt(p90))
    class_9 = classification_input(res).gte(p90)

    result_EA = class_0.multiply(1) \
                    .add(class_1.multiply(2)) \
                    .add(class_2.multiply(3)) \
                    .add(class_3.multiply(4)) \
                    .add(class_4.multiply(5)) \
                    .add(class_5.multiply(6)) \
                    .add(class_6.multiply(7)) \
                    .add(class_7.multiply(8)) \
                    .add(class_8.multiply(9)) \
                    .add(class_9.multiply(10)) \
                    .unmask(1)\
                    .updateMask(hansen_singleyear(widg_cal.value[0]).eq(1))\
                    .multiply(significance_buffer().unmask(0).neq(1))

    
    with status_widget:
        print('running ' + str(res) + 'm')
    
    return result_EA


In [26]:
def result_Jenks(res):
    
    randpoints = ee.FeatureCollection.randomPoints(region=geomX(), points=666, seed=7)
    input_Jenks_sampled = classification_input(res).sampleRegions(collection=randpoints, geometries=False)#.filter(ee.Filter.neq('distance',0)) # distance of 0 is non-forest
    value_list = input_Jenks_sampled.aggregate_array('treecover2000')
    jnb = JenksNaturalBreaks(10) # Asking for 10 clusters
    jnb.fit(value_list.getInfo()) # Create the clusters according to values in 'x'
    jib = jnb.inner_breaks_
    #print(jnb.inner_breaks_)
    
    class_0 = classification_input(res).lt(jib[0])
    class_1 = classification_input(res).gte(jib[0]).multiply(classification_input(res).lt(jib[1]))
    class_2 = classification_input(res).gte(jib[1]).multiply(classification_input(res).lt(jib[2]))
    class_3 = classification_input(res).gte(jib[2]).multiply(classification_input(res).lt(jib[3]))
    class_4 = classification_input(res).gte(jib[3]).multiply(classification_input(res).lt(jib[4]))
    class_5 = classification_input(res).gte(jib[4]).multiply(classification_input(res).lt(jib[5]))
    class_6 = classification_input(res).gte(jib[5]).multiply(classification_input(res).lt(jib[6]))
    class_7 = classification_input(res).gte(jib[6]).multiply(classification_input(res).lt(jib[7]))
    class_8 = classification_input(res).gte(jib[7]).multiply(classification_input(res).lt(jib[8]))
    class_9 = classification_input(res).gte(jib[8])

    result_jenks = class_0.multiply(1) \
                    .add(class_1.multiply(2)) \
                    .add(class_2.multiply(3)) \
                    .add(class_3.multiply(4)) \
                    .add(class_4.multiply(5)) \
                    .add(class_5.multiply(6)) \
                    .add(class_6.multiply(7)) \
                    .add(class_7.multiply(8)) \
                    .add(class_8.multiply(9)) \
                    .add(class_9.multiply(10)) \
                    .unmask(1)\
                    .updateMask(hansen_singleyear(widg_cal.value[0]).eq(1))\
                    .multiply(significance_buffer().unmask(0).neq(1))

    
    with status_widget:
        print('running ' + str(res) + 'm')
    
    return result_jenks
    


In [27]:
closed_forest_loss_single_year = ee.Image('projects/carbondevelopmentteam/assets/Philippines/PH004_Quirino_Protected_Landscape/risk_map_test/closed_forest_loss_single_year')

Map.addLayer(closed_forest_loss_single_year,{'min':0,'max':1,'palette':['white','firebrick']},'closed_forest_loss_single_year')

In [28]:
class_palette = ['fuchsia', '#008001', '#399c01', '#71b900', '#aad501', '#e3f000', '#fee300', '#ffaa01', '#ff7101', '#ff3900', '#fe0000']
class_viz = {'min':0, 'max':10, 'palette':class_palette}

In [29]:
#dummy = result_EA(600)

#Map.addLayer(dummy,class_viz,'result_EA_dummy')

In [30]:
#dummy_TESTER = result_EA_TESTER(600)

#Map.addLayer(dummy_TESTER,class_viz,'xTESTER result_EA_dummy')

In [31]:
#dummy_clin = classification_input(600)

#Map.addLayer(dummy_clin,class_viz,'dummy_clin')

In [32]:
def submit_Step3_clicked(b):
    # STEP 3: Creation of a risk map with discrete categories of risk.
    
    class_palette = ['fuchsia', '#008001', '#399c01', '#71b900', '#aad501', '#e3f000', '#fee300', '#ffaa01', '#ff7101', '#ff3900', '#fe0000']
    class_viz = {'min':0, 'max':10, 'palette':class_palette}

    Map.addLayer(result_EA(widg_class1.value),class_viz, 'Equal Area - ' + str(widg_class1.value) + 'm',True,0.0)
    Map.addLayer(result_EA(widg_class2.value),class_viz, 'Equal Area - ' + str(widg_class2.value) + 'm',True,0.0)
    Map.addLayer(result_EA(widg_class3.value),class_viz, 'Equal Area - ' + str(widg_class3.value) + 'm',True,0.0)
    Map.addLayer(result_EA(widg_class4.value),class_viz, 'Equal Area - ' + str(widg_class4.value) + 'm',True,0.0)
    Map.addLayer(result_EA(widg_class5.value),class_viz, 'Equal Area - ' + str(widg_class5.value) + 'm',True,0.0)
    Map.addLayer(result_EA(widg_class6.value),class_viz, 'Equal Area - ' + str(widg_class6.value) + 'm',True,0.0)
    Map.addLayer(result_EA(widg_class7.value),class_viz, 'Equal Area - ' + str(widg_class7.value) + 'm',True,0.0)
    Map.addLayer(result_EA(widg_class8.value),class_viz, 'Equal Area - ' + str(widg_class8.value) + 'm',True,0.0)
    Map.addLayer(result_EA(widg_class9.value),class_viz, 'Equal Area - ' + str(widg_class9.value) + 'm',True,0.0)
    Map.addLayer(result_EA(widg_class10.value),class_viz, 'Equal Area - ' + str(widg_class10.value) + 'm',True,0.0)

    layer_ea1 = Map.layers[-10]
    widg_ea1 = layer_ea1.interact(opacity=(0, 1, 0.1))
    layer_ea2 = Map.layers[-9]
    widg_ea2 = layer_ea2.interact(opacity=(0, 1, 0.1))
    layer_ea3 = Map.layers[-8]
    widg_ea3 = layer_ea3.interact(opacity=(0, 1, 0.1))
    layer_ea4 = Map.layers[-7]
    widg_ea4 = layer_ea4.interact(opacity=(0, 1, 0.1))
    layer_ea5 = Map.layers[-6]
    widg_ea5 = layer_ea5.interact(opacity=(0, 1, 0.1))
    layer_ea6 = Map.layers[-5]
    widg_ea6 = layer_ea6.interact(opacity=(0, 1, 0.1))
    layer_ea7 = Map.layers[-4]
    widg_ea7 = layer_ea7.interact(opacity=(0, 1, 0.1))
    layer_ea8 = Map.layers[-3]
    widg_ea8 = layer_ea8.interact(opacity=(0, 1, 0.1))
    layer_ea9 = Map.layers[-2]
    widg_ea9 = layer_ea9.interact(opacity=(0, 1, 0.1))
    layer_ea10 = Map.layers[-1]
    widg_ea10 = layer_ea10.interact(opacity=(0, 1, 0.1))
    
    label_ea1 = widgets.Label('Equal Area - ' + str(widg_class1.value) + 'm')
    label_ea2 = widgets.Label('Equal Area - ' + str(widg_class2.value) + 'm')
    label_ea3 = widgets.Label('Equal Area - ' + str(widg_class3.value) + 'm')
    label_ea4 = widgets.Label('Equal Area - ' + str(widg_class4.value) + 'm')
    label_ea5 = widgets.Label('Equal Area - ' + str(widg_class5.value) + 'm')
    label_ea6 = widgets.Label('Equal Area - ' + str(widg_class6.value) + 'm')
    label_ea7 = widgets.Label('Equal Area - ' + str(widg_class7.value) + 'm')
    label_ea8 = widgets.Label('Equal Area - ' + str(widg_class8.value) + 'm')
    label_ea9 = widgets.Label('Equal Area - ' + str(widg_class9.value) + 'm')
    label_ea10 = widgets.Label('Equal Area - ' + str(widg_class10.value) + 'm')

    full_widget = widgets.VBox([
        widgets.HBox([label_ea1, widg_ea1,label_ea2,widg_ea2,label_ea3, widg_ea3,label_ea4,widg_ea4]),
        widgets.HBox([label_ea5, widg_ea5,label_ea6,widg_ea6,label_ea7, widg_ea7,label_ea8,widg_ea8]),
        widgets.HBox([label_ea9, widg_ea9,label_ea10,widg_ea10]),
    ])
    
    tab_layers.children = tuple(list(tab_layers.children) + [full_widget])  
    tab_layers.set_title(2, 'Step 3')
    button_widget.children =  tuple(list(button_widget.children) + [submit_Step4]) 

submit_Step3.on_click(submit_Step3_clicked)

In [33]:
def makeGrid():
  # pixelLonLat returns an image with each pixel labeled with longitude and
  # latitude values.
  lonLat = ee.Image.pixelLonLat()

  # Select the longitude and latitude bands, multiply by a large number then
  # truncate them to integers.
  lonGrid = lonLat \
    .select('longitude') \
    .multiply(10000000) \
    .toInt()

  latGrid = lonLat \
    .select('latitude') \
    .multiply(10000000) \
    .toInt()

  # To produce the grid, multiply the latitude and longitude images and then use
  # reduce to vectors at the 10km resolution to group the grid into vectors.
  return lonGrid \
    .multiply(latGrid) \
    .reduceToVectors(
      geometry= geomX(), # This is undefined until you draw a geometry.
      scale= widg_gridres.value,
      geometryType= 'polygon'
    )


#Grids that do not contain any forest at the beginning of the validation period shall be excluded
#from the evaluation
# NOTE: NOT IN USE, as only useful in deserts


In [34]:
def all_forest():
    #we need the total area of forest at end of calibration / beginning of validation.
    forest_BeginningOfValidation = ee.Image.pixelArea().divide(10000).multiply(hansen_singleyear(widg_val.value[0])).reduceRegion(reducer=ee.Reducer.sum(), 
                                    geometry=geomX(), 
                                    scale=widg_res.value, 
                                    crs=ee.String('EPSG:4326'), 
                                    maxPixels=1e13)
    
    forest_BeginningOfValidation = ee.Number(forest_BeginningOfValidation.get(forest_BeginningOfValidation.keys().get(0)))

    return forest_BeginningOfValidation

#def results_by_classNum(num):
def wrapper_func(num,res):
    def xfunc(feat,featCol):

        #calibration before validation
        gridcell = feat.geometry()
        numYears_Validation = ee.Number(widg_val.value[1]).subtract(ee.Number(widg_val.value[0]))
        numYears_Calibration = ee.Number(widg_cal.value[1]).subtract(ee.Number(widg_cal.value[0]))
        
        pixel_area = ee.Image.pixelArea().divide(10000).multiply(result_EA(res).eq(num))

        #Column M
        forest_BeginningOfCalibration = pixel_area.multiply(hansen_singleyear(widg_cal.value[0])).reduceRegion(reducer=ee.Reducer.sum(), 
                                        geometry=gridcell, 
                                        scale=widg_res.value, 
                                        crs=ee.String('EPSG:4326'), 
                                        maxPixels=1e13)

        forest_BeginningOfCalibration = ee.Number(forest_BeginningOfCalibration.get(forest_BeginningOfCalibration.keys().get(0)))

        #Column N
        forest_BeginningOfValidation = pixel_area.multiply(hansen_singleyear(widg_val.value[0])).reduceRegion(reducer=ee.Reducer.sum(), 
                                        geometry=gridcell, 
                                        scale=widg_res.value, 
                                        crs=ee.String('EPSG:4326'), 
                                        maxPixels=1e13)


        forest_BeginningOfValidation = ee.Number(forest_BeginningOfValidation.get(forest_BeginningOfValidation.keys().get(0)))

        #Column K
        forest_EndOfValidation = pixel_area.multiply(hansen_singleyear(widg_val.value[1])).reduceRegion(reducer=ee.Reducer.sum(), 
                                        geometry=gridcell, 
                                        scale=widg_res.value, 
                                        crs=ee.String('EPSG:4326'), 
                                        maxPixels=1e13)


        forest_EndOfValidation = ee.Number(forest_EndOfValidation.get(forest_EndOfValidation.keys().get(0)))

        #Column L
        defor_Validation = forest_BeginningOfValidation.subtract(forest_EndOfValidation)

        #Column 0
        deforRate_Calibration = forest_BeginningOfValidation.subtract(forest_BeginningOfCalibration)

        #Column P
        #=(-N135*O135)*3
        defor_Predicted = deforRate_Calibration

        #Column Q
        grid_Weight = forest_BeginningOfValidation.divide(all_forest())

        #Column R
        #=((L2-P2)^2)*Q2
        column_R = defor_Validation.subtract(defor_Predicted).pow(2).multiply(grid_Weight)

        fc = ee.FeatureCollection([ee.Feature(None,{'column_R':column_R, 
                                                        'column_Q': grid_Weight,
                                                        'column_L': defor_Validation, 
                                                        'column_P': defor_Predicted,
                                                        'forest_BeginningOfValidation': forest_BeginningOfValidation,
                                                        'forest_BeginningOfCalibration': forest_BeginningOfCalibration,
                                                        'deforRate_Calibration': deforRate_Calibration,
                                                        'numYears_Validation': numYears_Validation,
                                                        'numYears_Calibration': numYears_Calibration
                                                   })])

        return ee.FeatureCollection(featCol).merge(fc)
    
    z = ee.FeatureCollection(makeGrid().iterate(xfunc,ee.FeatureCollection([])))
    with status_widget:
        print("...calculating...")
    return ee.FeatureCollection([z]).flatten()
    
def grid_Valued(res): 
    return ee.FeatureCollection([wrapper_func(1,res),
                                        wrapper_func(2,res),
                                        wrapper_func(3,res),
                                        wrapper_func(4,res),
                                        wrapper_func(5,res),
                                        wrapper_func(6,res),
                                        wrapper_func(7,res),
                                        wrapper_func(8,res),
                                        wrapper_func(9,res),
                                        wrapper_func(10,res),
                                           ]).flatten()



def wRMSE(res):
    sum_R = grid_Valued(res).reduceColumns(reducer=ee.Reducer.sum(), selectors=['column_R'])
    sum_R = ee.Number(sum_R.get(sum_R.keys().get(0)))

    sum_Q = grid_Valued(res).reduceColumns(reducer=ee.Reducer.sum(), selectors=['column_Q'])
    sum_Q = ee.Number(sum_Q.get(sum_Q.keys().get(0)))

    wRMSE = sum_R.divide(sum_Q).abs().sqrt()
    return wRMSE
    
    
# correlation between P & L
# NOTE: Not yet in use
def pearson(res):
    pearson = grid_Valued(res).reduceColumns(reducer=ee.Reducer.sum(), selectors=['forest_BeginningOfValidation'])

    #pearson = grid_Valued(res).reduceColumns(reducer=ee.Reducer.pearsonsCorrelation(), selectors=['column_L','column_P'])
    #pearson = ee.Number(pearson.get(pearson.keys().get(0)))
    return pearson

In [35]:
layout_widebutton = widgets.Layout(width='auto', height='40px') 
submit_report = widgets.Button(
    description='RUN REPORT',
    button_style='primary',
    tooltip='Click me',
    style=style,
    layout = layout_widebutton
)
submit_insignificant = widgets.Button(
    description='Download insignificant risk',
    button_style='primary',
    tooltip='Click me',
    style=style,
    layout = layout_widebutton
)
submit_forest = widgets.Button(
    description='Download forest cover & loss',
    button_style='primary',
    tooltip='Click me',
    style=style,
    layout = layout_widebutton
)
submit_class = widgets.Button(
    description='Download class maps',
    button_style='primary',
    tooltip='Click me',
    style=style,
    layout = layout_widebutton
)

download_hbox = widgets.HBox([submit_report, submit_insignificant, submit_forest, submit_class])

In [36]:
def submit_Step4_clicked(b):

    # STEP 4: Grid & rmse

    Map.addLayer(makeGrid(),{'color':'hotpink'}, 'Grid',True,0.0)

    with download_widget:
        display(download_hbox)

    layer_grid = Map.layers[-1]
    widg_grid = layer_grid.interact(opacity=(0, 1, 0.1))
    
    label_grid = widgets.Label('Grid')


    full_widget = widgets.VBox([
        widgets.HBox([label_grid, widg_grid]),
    ])
    
    

    tab_layers.children = tuple(list(tab_layers.children) + [full_widget])  
    tab_layers.set_title(3, 'Step 4')

 

submit_Step4.on_click(submit_Step4_clicked)

In [37]:
def submit_insignificant_clicked(b):
    with status_widget:
        print('Generating insignificant risk map for download...')


    export_grid = geomX().coveringGrid(ee.Projection("EPSG:4326"),40000)
    export_grid_list = ee.FeatureCollection(export_grid).toList(1500)
    
    
    for i in range(export_grid_list.length().getInfo()):
        insignificant_filename = os.path.join(out_dir, 'insignificant_'+ str(i) + '.tif')  

        export_gridcell_X = ee.Feature(export_grid_list.get(i)).geometry()
    
    
        raw_download = geemap.ee_export_image(ee_object=significance_buffer(), 
                        filename=insignificant_filename, 
                        scale=widg_res.value, 
                        crs='EPSG:4326', 
                        region=export_gridcell_X
                        )

        link = geemap.create_download_link(insignificant_filename,'Click here to download: ')

        with download_widget:
            display(link)
    
submit_insignificant.on_click(submit_insignificant_clicked)

In [38]:
def submit_forest_clicked(b):
    with status_widget:
        print('Generating forest map for download...')
    forest_filename = os.path.join(out_dir, 'forest.tif')  
    
    forest_band1 = hansen_singleyear(widg_cal.value[0])
    forest_band2 = historical_deforestation_NEW()
    
    forest_image = forest_band1.addBands(forest_band2).rename(['forest_cover','deforestation'])


    raw_download = geemap.ee_export_image(ee_object=forest_image, 
                    filename=forest_filename, 
                    scale=widg_res.value, 
                    crs='EPSG:4326', 
                    region=geomX()
                    )
    
    link = geemap.create_download_link(forest_filename,'Click here to download: ')
    
    with download_widget:
        display(link)
    
submit_forest.on_click(submit_forest_clicked)

In [39]:

def submit_class_clicked(b):
    with status_widget:
        print('Generating class map for download... This can take several minutes...')
    
    class_band2 = result_EA(widg_class2.value)
    #class_band2 = result_Jenks(widg_class2.value)

    
    class_image = class_band2

    
    export_grid = geomX().coveringGrid(ee.Projection("EPSG:4326"),40000)
    export_grid_list = ee.FeatureCollection(export_grid).toList(800)
    
    
    for i in range(export_grid_list.length().getInfo()):
        class_filename = os.path.join(out_dir, 'classes_'+ str(i) + '.tif')  

        export_gridcell_X = ee.Feature(export_grid_list.get(i)).geometry()
    
    
        raw_download = geemap.ee_export_image(ee_object=class_image, 
                        filename=class_filename, 
                        scale=widg_res.value, 
                        crs='EPSG:4326', 
                        region=export_gridcell_X
                        )

        link = geemap.create_download_link(class_filename,'Click here to download: ')

        with download_widget:
            display(link)
    
submit_class.on_click(submit_class_clicked)


In [40]:
'''
def submit_class_clicked(b):
    with status_widget:
        print('Generating class map for download... This can take several minutes...')
    
    class_band1 = result_EA(widg_class1.value)
    class_band2 = result_EA(widg_class2.value)
    class_band3 = result_EA(widg_class3.value)
    class_band4 = result_EA(widg_class4.value)
    class_band5 = result_EA(widg_class5.value)
    class_band6 = result_EA(widg_class6.value)
    class_band7 = result_EA(widg_class7.value)
    class_band8 = result_EA(widg_class8.value)
    class_band9 = result_EA(widg_class9.value)
    class_band10 = result_EA(widg_class10.value)
    
    class_image = class_band1\
                    .addBands(class_band2)\
                    .addBands(class_band3)\
                    .addBands(class_band4)\
                    .addBands(class_band5)\
                    .addBands(class_band6)\
                    .addBands(class_band7)\
                    .addBands(class_band8)\
                    .addBands(class_band9)\
                    .addBands(class_band10)\
                    .rename(['class1',
                             'class2',
                             'class3',
                             'class4',
                             'class5',
                             'class6',
                             'class7',
                             'class8',
                             'class9',
                             'class10',
                            ])

    
    export_grid = geomX().coveringGrid(ee.Projection("EPSG:4326"),40000)
    export_grid_list = ee.FeatureCollection(export_grid).toList(50)
    
    
    for i in range(5,8):#export_grid_list.length().getInfo()):
        class_filename = os.path.join(out_dir, 'classes_'+ str(i) + '.tif')  

        export_gridcell_X = ee.Feature(export_grid_list.get(i)).geometry()
    
    
        raw_download = geemap.ee_export_image(ee_object=class_image, 
                        filename=class_filename, 
                        scale=widg_res.value, 
                        crs='EPSG:4326', 
                        region=export_gridcell_X
                        )

        link = geemap.create_download_link(class_filename,'Click here to download: ')

        with download_widget:
            display(link)
    
submit_class.on_click(submit_class_clicked)

'''

'\ndef submit_class_clicked(b):\n    with status_widget:\n        print(\'Generating class map for download... This can take several minutes...\')\n    \n    class_band1 = result_EA(widg_class1.value)\n    class_band2 = result_EA(widg_class2.value)\n    class_band3 = result_EA(widg_class3.value)\n    class_band4 = result_EA(widg_class4.value)\n    class_band5 = result_EA(widg_class5.value)\n    class_band6 = result_EA(widg_class6.value)\n    class_band7 = result_EA(widg_class7.value)\n    class_band8 = result_EA(widg_class8.value)\n    class_band9 = result_EA(widg_class9.value)\n    class_band10 = result_EA(widg_class10.value)\n    \n    class_image = class_band1                    .addBands(class_band2)                    .addBands(class_band3)                    .addBands(class_band4)                    .addBands(class_band5)                    .addBands(class_band6)                    .addBands(class_band7)                    .addBands(class_band8)                    .addBands(class

In [41]:
def submit_report_clicked(b):
    with status_widget:
        print('Generating report... This can take several minutes...')
        
    today = date.today()

    pdf = FPDF()
    pdf.add_page()
    pdf.set_xy(0, 0)
    pdf.set_font('arial', 'B', 12)
    pdf.cell(60)
    pdf.cell(75, 10, "Hectares: " + 'x', 0, 2, 'C')
    pdf.cell(75, 10, "Date: " + str(today), 0, 2, 'C')

    pdf.cell(90, 10, " ", 0, 2, 'C')
    pdf.cell(-60)

    pdf.set_font('arial', 'B', 12)
    pdf.cell(60)
    pdf.cell(75, 10, "Stats", 0, 2, 'C')
    pdf.cell(90, 10, " ", 0, 2, 'C')
    pdf.cell(-40)
    pdf.cell(40, 10, 'Resolution', 1, 0, 'C')
    pdf.cell(40, 10, 'wRMS', 1, 0, 'C')
    pdf.cell(40, 10, 'Correl', 1, 0, 'C')
    pdf.cell(40, 10, 'R2', 1, 2, 'C')
    pdf.cell(-120)
    pdf.set_font('arial', '', 10)
    '''
    pdf.cell(40, 10, str(widg_class1.value) + 'm', 1, 0, 'C')
    pdf.cell(40, 10, str(round(wRMSE(widg_class1.value).getInfo(),2)), 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 2, 'C')
    pdf.cell(-120)
    pdf.cell(40, 10, str(widg_class2.value) + 'm', 1, 0, 'C')
    pdf.cell(40, 10, str(round(wRMSE(widg_class2.value).getInfo(),2)), 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 2, 'C')
    pdf.cell(-120)
    pdf.cell(40, 10, str(widg_class3.value) + 'm', 1, 0, 'C')
    pdf.cell(40, 10, str(round(wRMSE(widg_class3.value).getInfo(),2)), 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 2, 'C')
    pdf.cell(-120)
    pdf.cell(40, 10, str(widg_class4.value) + 'm', 1, 0, 'C')
    pdf.cell(40, 10, str(round(wRMSE(widg_class4.value).getInfo(),2)), 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 2, 'C')
    pdf.cell(-120)
    pdf.cell(40, 10, str(widg_class5.value) + 'm', 1, 0, 'C')
    pdf.cell(40, 10, str(round(wRMSE(widg_class5.value).getInfo(),2)), 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 2, 'C')
    pdf.cell(-120)
    pdf.cell(40, 10, str(widg_class6.value) + 'm', 1, 0, 'C')
    pdf.cell(40, 10, str(round(wRMSE(widg_class6.value).getInfo(),2)), 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 2, 'C')
    pdf.cell(-120)
    pdf.cell(40, 10, str(widg_class7.value) + 'm', 1, 0, 'C')
    pdf.cell(40, 10, str(round(wRMSE(widg_class7.value).getInfo(),2)), 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 2, 'C')
    pdf.cell(-120)
    pdf.cell(40, 10, str(widg_class8.value) + 'm', 1, 0, 'C')
    pdf.cell(40, 10, str(round(wRMSE(widg_class8.value).getInfo(),2)), 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 2, 'C')
    pdf.cell(-120)
    '''
    pdf.cell(40, 10, str(widg_class9.value) + 'm', 1, 0, 'C')
    pdf.cell(40, 10, str(round(wRMSE(widg_class9.value).getInfo(),2)), 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 2, 'C')
    pdf.cell(-120)
    pdf.cell(40, 10, str(widg_class10.value) + 'm', 1, 0, 'C')
    pdf.cell(40, 10, str(round(wRMSE(widg_class10.value).getInfo(),2)), 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 0, 'C')
    pdf.cell(40, 10, '', 1, 2, 'C')

    
    
    output = pdf.output(pdf_link, 'F')

    
    link = geemap.create_download_link(pdf_link, title="Click here to download report: ")
    with download_widget:
        display(link)

    
submit_report.on_click(submit_report_clicked)

[Make a suggestion!](https://forms.gle/cm5qoDKG3crohtqZA)

In [42]:
#PRINT DISTANCE FOR SIGNIFICANCE
randpoints = ee.FeatureCollection.randomPoints(region=geomX(), points=10000, seed=7)

dist_sampled = dist_edge().updateMask(historical_deforestation_NEW()).sampleRegions(collection=randpoints,geometries=False)

sampled_pxx = dist_sampled.reduceColumns(reducer=ee.Reducer.percentile([widg_percentile.value]), selectors=['distance'])
sampled_pxx = ee.Number(sampled_pxx.get(sampled_pxx.keys().get(0)))

print(sampled_pxx.getInfo())

not enough values to unpack (expected 1, got 0)


EEException: Unrecognized argument type to convert to a FeatureCollection: None

In [ ]:
print(jnb.breaks_) # Break values (including min and max)

In [ ]:
print(jnb.inner_breaks_) # Inner breaks (ie breaks_[1:-1])

In [ ]:
#def result_Jenks(res):
res = 1200
randpoints = ee.FeatureCollection.randomPoints(region=geomX(), points=666, seed=7)
input_Jenks_sampled = classification_input(res).sampleRegions(collection=randpoints, geometries=False)#.filter(ee.Filter.neq('distance',0)) # distance of 0 is non-forest
value_list = input_Jenks_sampled.aggregate_array('treecover2000')

#keys = input_Jenks_sampled.first().toDictionary().keys()
#print(value_list.getInfo())
    
#print(value_list.getInfo())
    

#x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
jnb = JenksNaturalBreaks(10) # Asking for 4 clusters

jnb.fit(value_list.getInfo()) # Create the clusters according to values in 'x'
#print(jnb.labels_) # Labels for fitted data
#print(jnb.inner_breaks_)
print(jnb.inner_breaks_[2])


'''

var features = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")
var keys = features.first().toDictionary().keys()
var attributes = features.map(function (f) {
  return features.set('dict', f.toDictionary(keys));
});
print(attributes.aggregate_array('dict'))



'''


In [ ]:
res = 3000
randpoints = ee.FeatureCollection.randomPoints(region=geomX(), points=10, seed=7)
input_EA_sampled = classification_input(res).sampleRegions(collection=randpoints, geometries=False)#.filter(ee.Filter.neq('distance',0)) # distance of 0 is non-forest
print(input_EA_sampled.getInfo())